# Beta Bank 

## Introduction

In the dynamic retail banking landscape, customer retention holds paramount importance for sustained growth and profitability. Beta Bank is witnessing a concerning trend: a gradual erosion of its customer base, leading to a significant impact on its financial performance. Recognizing the urgency of the situation, the bank has launched a project aimed at developing a predictive model capable of anticipating customer churn.

The objective of this project is to utilize historical data on customer behavior and termination patterns to forecast the likelihood of customer attrition in the near future. By accurately identifying at-risk customers, Beta Bank can proactively implement retention strategies to safeguard its customer base and optimize operational efficiency.

The primary performance metric for evaluating this predictive model is the F1 score, with a minimum threshold of 0.59 required for project approval. The F1 score, striking a balance between precision and recall, ensures reliable churn identification.

Additionally, the model's performance will be assessed using the AUC-ROC (Area Under the Receiver Operating Characteristic curve) metric. A high AUC-ROC value indicates the model's effectiveness in distinguishing between churn and non-churn instances across various decision thresholds.

Through this project, the goal is to deliver a robust predictive tool that empowers Beta Bank to preemptively address customer churn, fostering enduring customer relationships and sustaining the bank's competitive edge in the market. This initiative represents a strategic step towards enhancing Beta Bank's operational resilience and ensuring long-term success in the evolving banking landscape.

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, f1_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.utils import shuffle, resample

pd.options.mode.chained_assignment = None

In [2]:
try:
    columns = ['row_number', 'customer_id', 'surname', 'credit_score', 'geography', 'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card', 'is_active_member', 'estimated_salary', 'exited']
    data = pd.read_csv('/datasets/Churn.csv', header=0, names=columns)
except FileNotFoundError as e:
    print(f"The error ({e}) occured while trying to load the data")
else:
    print(f"The data was loaded successfully!")

The data was loaded successfully!


## Data Preparation

In [3]:
data.shape

(10000, 14)

In [4]:
# Look at data types and missing values

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            9091 non-null   float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


The columns have the appropriate data types, although it appears that the tenure column contains missing values. The missing values will be filled using the median, because of the wide range of values. 

In [5]:
num_duplicated = data.duplicated().sum()
num_missing = data['tenure'].isna().sum()

print(f"Number of duplicated values: {num_duplicated}\nNumber of missing values in 'tenure' column: {num_missing}")

Number of duplicated values: 0
Number of missing values in 'tenure' column: 909


In [6]:
# Observe random sample of the rows with missing values in 'tenure' col

data[data['tenure'].isna()].sample(10)

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
9130,9131,15790599,Yin,756,Germany,Female,39,NaN,149363.12,2,1,1,109098.39,0
1342,1343,15640268,Avdeeva,652,Spain,Male,71,NaN,0.00,1,1,1,120107.10,0
597,598,15567446,Coffman,646,Germany,Male,39,NaN,111574.41,1,1,1,30838.51,0
3510,3511,15800814,Palerma,534,France,Male,35,NaN,81951.74,2,1,0,115668.53,0
660,661,15592937,Napolitani,632,Germany,Female,41,NaN,81877.38,1,1,1,33642.21,0
4428,4429,15742028,Udegbulam,602,France,Female,35,NaN,0.00,2,1,0,31050.02,0
3436,3437,15735632,Williamson,571,France,Male,41,NaN,0.00,1,1,1,63736.17,0
5889,5890,15708650,Fullwood,727,France,Female,31,NaN,52192.08,2,0,1,160383.47,0
8432,8433,15582519,Seleznyov,479,France,Male,47,NaN,121797.09,1,0,1,5811.90,1
3486,3487,15630661,Vasilyev,614,Spain,Female,25,NaN,75212.28,1,1,0,58965.04,0


In [7]:
# Observe random sample of the rows WITHOUT missing values in 'tenure' col

data[data['tenure'] != 'NaN'].sample(10)

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
9195,9196,15710315,Chukwukadibia,529,Germany,Male,29,4.0,135759.40,1,0,0,112813.79,1
4787,4788,15752885,Nnonso,529,France,Male,42,1.0,157498.90,1,1,1,82276.62,0
3251,3252,15750335,Paterson,850,Germany,Male,43,0.0,108508.82,3,1,0,184044.80,1
9109,9110,15703682,Kalinina,681,Spain,Male,33,NaN,0.00,1,0,0,158336.36,0
5800,5801,15610379,Barclay-Harvey,599,France,Male,30,9.0,105443.68,1,1,1,121124.53,0
7781,7782,15759184,Russell,705,France,Male,34,7.0,117715.84,1,1,0,2498.67,0
1704,1705,15739054,Y?,654,France,Female,29,4.0,96974.97,1,0,1,141404.07,0
9183,9184,15676091,Iloerika,543,France,Male,42,NaN,0.00,1,1,1,56650.47,0
2297,2298,15797595,Greenhalgh,709,France,Female,40,9.0,131569.63,1,1,1,103970.58,0
7937,7938,15775886,Su,670,France,Male,36,NaN,0.00,1,1,0,140754.19,1


In [8]:
# Fill missing values with median
median_value = data['tenure'].median()
data['tenure'].fillna(median_value, inplace=True)

# Reprint number of missing values
num_missing = data['tenure'].isna().sum()
print(f"Number of missing values in 'tenure' column: {num_missing}")

Number of missing values in 'tenure' column: 0


## Feature Preparation

In [9]:
# Drop rows that will not be relevant to the model

data = data.drop(['row_number', 'customer_id', 'surname'], axis=1)

In [10]:
# Find unique values in 'gender' col
gen_unique = data['gender'].unique()

# Find unique values in 'geography' col
geo_unique = data['geography'].unique()

print(f"Unique values in the geography column:{geo_unique}\nUnique values in the gender column:{gen_unique}")

Unique values in the geography column:['France' 'Spain' 'Germany']
Unique values in the gender column:['Female' 'Male']


The geography and gender columns are of the object data type, which will not work with machine learning. These values need to be converted to a numerical value, using one-hot encoding that can be used in machine learning. 

In [11]:
# Grab columns with 'object' data type
object_cols = data.select_dtypes(include=['object']).columns

data = pd.get_dummies(data, columns=object_cols, drop_first=True)

Since only one dataset is available, the source data will be divided into a split of 3:1:1. This allocation designates 60% of the data for training, while 20% is reserved for both validation and testing purposes.

In [12]:
# Function to split data into 3:1:1 ratio and return the three datasets

def split_data_3_1_1(dataset, test_s=0.40, rnd_state=42, shuffle=True):
    '''Prints a statement specifying the data-split used and returns 3 variables for the train, validation and test datasets respectively'''
    
    # Splitting the source data into 40% for Validation (to be split again) and 60% for Training
    df_train, df_valid = train_test_split(dataset, test_size=test_s, shuffle=True, random_state=rnd_state)

    # Further splitting the df_valid data 50/50 (40% from previous task) to obtain 3:1:1 ratio
    df_test, df_valid = train_test_split(df_valid, test_size=(test_s + .10), shuffle=True, random_state=rnd_state)
    
    # Printing confirmation of data split
    sum_of_datasets = len(df_train) + len(df_valid) + len(df_test)
    if len(data) == sum_of_datasets:
        print(f"Data split ratio is 3:1:1, where data split is allocated as:\nTraining = 60% [shape={df_train.shape}]\nValidation = 20% [shape={df_valid.shape}]\nTesting = 20% [shape={df_test.shape}]")
    
    return df_train, df_valid, df_test


# Function to declare the features and target from dataset
def prepare_data(dataset, drop_cols, target_col):
    '''Returns the features and target for a given dataset'''
    features = dataset.drop(drop_cols, axis=1)
    target = dataset[target_col]
    
    return features, target

In [13]:
# Create train, validation and test dataset
df_train, df_valid, df_test = split_data_3_1_1(data, 0.40, 24681)

# Declare the train, validation, and test features & target
train_features, train_target = prepare_data(df_train, ['exited'], 'exited')
valid_features, valid_target = prepare_data(df_valid, ['exited'], 'exited')
test_features, test_target = prepare_data(df_test, ['exited'], 'exited')

Data split ratio is 3:1:1, where data split is allocated as:
Training = 60% [shape=(6000, 12)]
Validation = 20% [shape=(2000, 12)]
Testing = 20% [shape=(2000, 12)]


In [14]:
# Observe the train dataset

df_train.head(5)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
8384,755,34,3.0,0.00,2,1,1,158816.03,0,0,0,1
2187,699,34,8.0,0.00,1,1,1,76510.46,0,0,1,1
2974,565,59,9.0,69129.59,1,1,1,170705.53,0,1,0,1
4760,850,40,1.0,76914.21,1,1,0,174183.44,0,0,0,1
9814,631,36,1.0,0.00,2,0,0,133141.34,0,0,0,1


In [15]:
# Observe the validation dataset

df_valid.head(5)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
1892,634,31,8.0,76798.92,1,0,0,196021.73,0,1,0,0
6724,752,41,8.0,0.00,2,1,0,139844.04,1,0,0,0
1558,571,35,1.0,104783.81,2,0,1,178512.52,0,0,0,0
7100,650,24,5.0,108881.73,1,1,0,104492.83,0,0,1,1
329,664,26,5.0,116244.14,2,1,1,95145.14,0,1,0,1


In [16]:
# Observe the test dataset

df_test.head(5)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
9254,686,32,6.0,0.00,2,1,1,179093.26,0,0,0,1
5088,543,42,4.0,89838.71,3,1,0,85983.54,1,0,0,1
2516,814,36,6.0,0.00,2,1,1,98657.01,0,0,0,0
1834,700,46,5.0,56580.95,2,0,1,45424.13,0,0,1,0
622,535,53,5.0,141616.55,2,1,1,75888.65,0,1,0,0


The numeric columns in the datasets exhibit a diverse range of values. The credit_score, age, tenure, balance, num_of_products, and estimated_salary columns all display a wide spectrum of values. To address this variability, these columns require standardization using a scaler.

In [18]:
# Fit the features on the three datasets
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']
scaler = StandardScaler()

scaler.fit(train_features[numeric])

train_features[numeric] = scaler.transform(train_features[numeric])
valid_features[numeric] = scaler.transform(valid_features[numeric])
test_features[numeric] = scaler.transform(test_features[numeric])

In [20]:
# Re-examine the features after sclaing
valid_features.head(5)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
1892,-0.170053,-0.746397,1.074465,0.001407,-0.904524,0,0,1.678152,1,0,0
6724,1.061746,0.194202,1.074465,-1.236040,0.833831,1,0,0.701775,0,0,0
1558,-0.827708,-0.370157,-1.453547,0.452322,0.833831,0,1,1.373839,0,0,0
7100,-0.003029,-1.404817,-0.008968,0.518351,-0.904524,1,0,0.087366,0,1,1
329,0.143117,-1.216697,-0.008968,0.636980,0.833831,1,1,-0.075098,1,0,1
